In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jaxlie import SE3, SO3
import jax_dataclasses as jdc
from functools import partial
import PyCeres

from sdf_world.sdf_world import *
from sdf_world.robots import *
from sdf_world.util import *

In [2]:
world = SDFWorld()
panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)
panda = Robot(world.vis, "panda", panda_model, alpha=0.5)
panda.reduce_dim([7, 8], [0.04, 0.04])

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
#utility
def to_posevec(pose:SE3):
    return jnp.hstack([pose.translation(), pose.rotation().log()])

def make_pose():
    return SE3.from_rotation_and_translation(
        SO3(np.random.random(4)).normalize(),
        np.random.uniform([-0.3,-0.5,0.3],[0.6, 0.5, 0.8])
    )

# Kinematics
def get_rotvec_angvel_map(v):
    def skew(v):
        v1, v2, v3 = v
        return jnp.array([[0, -v3, v2],
                        [v3, 0., -v1],
                        [-v2, v1, 0.]])
    vmag = jnp.linalg.norm(v)
    vskew = skew(v)
    return jnp.eye(3) \
        - 1/2*skew(v) \
        + vskew@vskew * 1/vmag**2 * (1-vmag/2 * jnp.sin(vmag)/(1-jnp.cos(vmag)))

@jax.jit
def get_ee_fk_jac(q):
    # outputs ee_posevec and analytical jacobian
    fks = panda_model.fk_fn(q)
    p_ee = fks[-1][-3:]
    rotvec_ee = SO3(fks[-1][:4]).log()
    E = get_rotvec_angvel_map(rotvec_ee)
    jac = []
    for posevec in fks[1:8]:
        p_frame = posevec[-3:]
        rot_axis = SE3(posevec).as_matrix()[:3, 2]
        lin_vel = jnp.cross(rot_axis, p_ee - p_frame)
        jac.append(jnp.hstack([lin_vel, rot_axis]))
    jac = jnp.array(jac).T
    jac = jac.at[3:, :].set(E @ jac[3:, :])
    return jnp.hstack([p_ee, rotvec_ee]), jac



In [4]:
frame = Frame(world.vis, "frame")

In [23]:
import time
class Kin(PyCeres.CostFunction):
    def __init__(self):
        super().__init__()
        self.set_num_residuals(6)
        self.set_parameter_block_sizes([7])
        self.goal = None
        self.weight_mat = None

    def set_goal(self, posevec):
        self.goal = posevec
    
    def set_weight(self, weight):
        self.weight_mat = np.diag(np.sqrt(weight))

    def Evaluate(self, parameters, residuals, jacobians):
        q = parameters[0] #vector
        ee, jac = get_ee_fk_jac(q)
        residuals[:] = self.weight_mat @ (self.goal - ee)
        panda.set_joint_angles(q)
        time.sleep(0.05)
        if (jacobians != None):
            jacobians[0][:] = - (self.weight_mat @ jac).flatten()
        return True

In [29]:
pose_d = make_pose()
frame.set_pose(pose_d)
posevec_d = to_posevec(pose_d)

In [24]:
pose_weight = np.array([1, 1, 1, 0.3, 0.3, 0.3])

In [30]:
feature_kin = Kin()
feature_kin.set_goal(posevec_d)
feature_kin.set_weight(pose_weight)

In [26]:
# residuals = np.zeros(6)
# jacobians = [np.zeros(42)]
#feature_kin.Evaluate([q], residuals, jacobians) # TEST

In [31]:
#initialize
q = panda.neutral.copy()
panda.set_joint_angles(q)

problem = PyCeres.Problem()
problem.AddResidualBlock(feature_kin, None, q)
options = PyCeres.SolverOptions()
options.linear_solver_type = PyCeres.LinearSolverType.DENSE_QR
options.parameter_tolerance = 1e-2
options.minimizer_progress_to_stdout = True
summary = PyCeres.Summary()

In [32]:
PyCeres.Solve(options, problem, summary)

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.821009e-01    0.00e+00    5.26e-01   0.00e+00   0.00e+00  1.00e+04        0    5.60e-02    5.61e-02
   1  6.132471e-02    2.21e-01    1.11e-01   2.49e+00   7.83e-01  1.22e+04        1    1.10e-01    1.66e-01
   2  5.325204e+00   -5.26e+00    1.11e-01   1.50e+00  -8.58e+01  6.10e+03        1    6.87e-02    2.35e-01
   3  5.328043e+00   -5.27e+00    1.11e-01   1.50e+00  -8.59e+01  1.53e+03        1    5.54e-02    2.90e-01
   4  5.344507e+00   -5.28e+00    1.11e-01   1.48e+00  -8.62e+01  1.91e+02        1    5.31e-02    3.44e-01
   5  5.461678e+00   -5.40e+00    1.11e-01   1.35e+00  -8.87e+01  1.19e+01        1    5.37e-02    3.97e-01
   6  1.988135e-02    4.14e-02    4.06e-02   6.23e-01   9.66e-01  3.58e+01        1    1.15e-01    5.12e-01
   7  3.240180e-03    1.66e-02    1.77e-02   5.89e-01   9.52e-01  1.07e+02        1    1.15e-01    6.27e-01
   8  2.589624e-04    2.98e-

In [22]:
panda.set_joint_angles(q)

In [104]:
q

array([ 0.42615311, -0.48190832, -1.13909383, -2.00997517, -3.79805377,
        2.01518086,  1.47913683])

In [78]:
summary.total_time_in_seconds

0.019966840744018555

In [18]:
def powell_fn(x):
    x1, x2, x3, x4 = x
    f1 = x1 + 10*x2
    f2 = jnp.sqrt(5)*(x3-x4)
    f3 = (x2 - 2*x3)**2
    f4 = jnp.sqrt(10)*(x1 - x4)**2
    return jnp.hstack([f1, f2, f3, f4])
jac_powell = jax.jacrev(powell_fn)
jac_powell(jnp.zeros(4))

In [76]:
print(summary.BriefReport())

Ceres Solver Report: Iterations: 22, Initial cost: 1.169696e+00, Final cost: 9.534040e-15, Termination: CONVERGENCE


In [38]:
feature = Powell()

In [39]:
data = jnp.array([3, -1, 0, 1.])

In [42]:
PyCeres.Solve(options, problem, summary)

[ 3. -1.  0.  1.] :)
iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.075000e+02    0.00e+00    1.55e+02   0.00e+00   0.00e+00  1.00e+04        0    2.30e-02    2.30e-02
[ 1.20093009 -0.11998137  0.1925488   0.20075604] :)
[ 1.20093009 -0.11998137  0.1925488   0.20075604] :)
   1  5.036190e+00    1.02e+02    2.00e+01   2.16e+00   9.53e-01  3.00e+04        1    2.09e-02    4.39e-02
[ 0.59696117 -0.05969324  0.09666367  0.09685423] :)
[ 0.59696117 -0.05969324  0.09666367  0.09685423] :)
   2  3.148168e-01    4.72e+00    2.50e+00   6.23e-01   9.37e-01  9.00e+04        1    2.24e-02    6.62e-02
[ 0.29844663 -0.02984475  0.04838032  0.04838941] :)
[ 0.29844663 -0.02984475  0.04838032  0.04838941] :)
   3  1.967760e-02    2.95e-01    3.13e-01   3.08e-01   9.37e-01  2.70e+05        1    2.14e-02    8.76e-02
[ 0.14923328 -0.01492337  0.02420314  0.02420374] :)
[ 0.14923328 -0.01492337  0.02420314  0.02420374] :)
   4  1.229900e

In [36]:
print(summary.BriefReport())

Ceres Solver Report: Iterations: 15, Initial cost: 1.075000e+02, Final cost: 1.120032e-15, Termination: CONVERGENCE


In [2]:
world = SDFWorld()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)
panda = Robot(world.vis, "panda", panda_model, alpha=0.5)
panda.reduce_dim([7, 8], [0.04, 0.04])

In [4]:
initial_x = 5.0

In [5]:
x = np.array([initial_x])

In [6]:
problem = PyCeres.Problem()

In [14]:
import jax

In [ ]:
jax.grad()

In [8]:
class QuadraticCostFunction(PyCeres.CostFunction):
    def __init__(self):
        super().__init__()
        self.set_num_residuals(1)
        self.set_parameter_block_sizes([1])
    def Evaluate(self, parameters, residuals, jacobians):
        x = parameters[0][0]
        residuals[0] = 10 - x
        if (jacobians != None) :
            jacobians[0][0] = -1
        return True
quad_cost_fun = QuadraticCostFunction()

In [9]:
problem.AddResidualBlock(quad_cost_fun, None, x)

In [10]:
options = PyCeres.SolverOptions()
options.linear_solver_type = PyCeres.LinearSolverType.DENSE_QR
options.minimizer_progress_to_stdout = True
summary = PyCeres.Summary()

In [12]:
PyCeres.Solve(options, problem, summary)

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.250000e+01    0.00e+00    5.00e+00   0.00e+00   0.00e+00  1.00e+04        0    2.16e-04    3.07e-04
   1  1.249750e-07    1.25e+01    5.00e-04   5.00e+00   1.00e+00  3.00e+04        1    2.72e-04    6.02e-04
   2  1.388518e-16    1.25e-07    1.67e-08   5.00e-04   1.00e+00  9.00e+04        1    3.51e-04    9.62e-04


In [13]:
print(summary.BriefReport() + " \n")
print("x : " + str(initial_x) + " -> " + str(x) + "\n")

Ceres Solver Report: Iterations: 3, Initial cost: 1.250000e+01, Final cost: 1.388518e-16, Termination: CONVERGENCE 

x : 5.0 -> [9.99999998]

